# Task 2 (Detecting depressed subjects)
### Using contextualized language models

In [1]:
P_FILE = "../posts.csv"
TRAIN_TOKEN="../train_df.csv"
TEST_TOKEN="../test_df.csv"
GENERAL_MODELS="../Models"
ROLLING_WINDOW_SIZE=10
LM_MODEL="all-mpnet-base-v2"#'all-MiniLM-L6-v2'
CONVERTED=True
max_lengths={1:64,3:128,5:256,10:512}
MODEL_PATH =f"{GENERAL_MODELS}/LM/NN_win_{ROLLING_WINDOW_SIZE}" 



In [2]:
from pathlib import Path
Path(MODEL_PATH).mkdir(parents=True, exist_ok=True)

### opening resulting dataset with pandas

In [3]:
import pandas as pd
import numpy as np
import os
seed=23
np.random.seed(seed)

train_df = pd.read_pickle(f"train_df_{ROLLING_WINDOW_SIZE}_sent.pkl")
test_df = pd.read_pickle(f"test_df_{ROLLING_WINDOW_SIZE}_sent.pkl")
train_df

,User,Window_id,Text,Label,polarity,subjectivity,negativity,positivity,neutrality,compound
0,test_subject1345,0,so many unwanted smith fadeaways. mid range j...,1,0.663051,0.516980,0.053,0.145,0.802,0.98315
1,test_subject1345,1,"mid range jumpers hey guys, celtics fan here p...",1,0.650092,0.517633,0.048,0.147,0.805,0.98440
2,test_subject1345,2,well he got number tonight so maybe he will b...,1,0.588214,0.594881,0.082,0.121,0.797,0.75595
3,test_subject1345,3,i mean he will get pinch hits and an occasion...,1,0.573929,0.588929,0.088,0.107,0.805,0.60815
4,test_subject1345,4,yeah you are probably right. oh well. i gues...,1,0.626531,0.626531,0.091,0.118,0.792,0.63660
...,...,...,...,...,...,...,...,...,...,...
174168,subject9959,627,nothing like that clean house feeling there i...,0,0.834343,0.551515,0.097,0.033,0.869,0.28280
174169,subject9959,628,there is always that one coworker... there is...,0,0.818182,0.477273,0.000,0.000,1.000,0.50000
174170,subject9959,629,there is always that one coworker you just can...,0,0.818182,0.477273,0.000,0.000,1.000,0.50000
174171,subject9959,630,that moment when you realize you need a new jo...,0,0.818182,0.477273,0.000,0.000,1.000,0.50000


In [4]:
import pickle
#Store sentences & embeddings on disc
def save_embeddings(filepath, embeddings):
    with open(filepath, "wb") as fOut:
        pickle.dump({ 'embeddings': embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

#Load sentences & embeddings from disc
def load_embeddings(filepath):
    with open(filepath, "rb") as fIn:
        stored_data = pickle.load(fIn)
        stored_embeddings = stored_data['embeddings']
    return stored_embeddings   

In [5]:

if not CONVERTED:
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer(LM_MODEL)

    model.max_seq_length = max_lengths[ROLLING_WINDOW_SIZE]
    train_sentence_embeddings = model.encode(train_df['Text'],show_progress_bar=True,\
                output_value='sentence_embedding', batch_size=64,convert_to_numpy=True)

    val_sentence_embeddings = model.encode(test_df['Text'],show_progress_bar=True,\
                output_value='sentence_embedding', batch_size=64,convert_to_numpy=True)

    save_embeddings(f"./train_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl",train_sentence_embeddings)
    save_embeddings(f"./val_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl",val_sentence_embeddings)
else:
    train_sentence_embeddings = load_embeddings(f"./train_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")
    val_sentence_embeddings = load_embeddings(f"./val_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")
   
test_df['Vector'] = pd.DataFrame(data=val_sentence_embeddings).values.tolist()
train_df['Vector'] = pd.DataFrame(data=train_sentence_embeddings).values.tolist()

In [6]:
train_df

,User,Window_id,Text,Label,polarity,subjectivity,negativity,positivity,neutrality,compound,Vector
0,test_subject1345,0,so many unwanted smith fadeaways. mid range j...,1,0.663051,0.516980,0.053,0.145,0.802,0.98315,"[-0.054277509450912476, 0.0824999287724495, 0...."
1,test_subject1345,1,"mid range jumpers hey guys, celtics fan here p...",1,0.650092,0.517633,0.048,0.147,0.805,0.98440,"[-0.054094888269901276, 0.06312193721532822, 0..."
2,test_subject1345,2,well he got number tonight so maybe he will b...,1,0.588214,0.594881,0.082,0.121,0.797,0.75595,"[-0.047845326364040375, 0.0940595269203186, -0..."
3,test_subject1345,3,i mean he will get pinch hits and an occasion...,1,0.573929,0.588929,0.088,0.107,0.805,0.60815,"[-0.04452856630086899, 0.09207145869731903, -0..."
4,test_subject1345,4,yeah you are probably right. oh well. i gues...,1,0.626531,0.626531,0.091,0.118,0.792,0.63660,"[-0.03719930350780487, 0.08828624337911606, -0..."
...,...,...,...,...,...,...,...,...,...,...,...
174168,subject9959,627,nothing like that clean house feeling there i...,0,0.834343,0.551515,0.097,0.033,0.869,0.28280,"[-0.014173840172588825, 0.08119028061628342, 0..."
174169,subject9959,628,there is always that one coworker... there is...,0,0.818182,0.477273,0.000,0.000,1.000,0.50000,"[0.006969124544411898, 0.08165775239467621, -0..."
174170,subject9959,629,there is always that one coworker you just can...,0,0.818182,0.477273,0.000,0.000,1.000,0.50000,"[-0.014388704672455788, 0.09135324507951736, -..."
174171,subject9959,630,that moment when you realize you need a new jo...,0,0.818182,0.477273,0.000,0.000,1.000,0.50000,"[-0.0011959681287407875, 0.09174152463674545, ..."


In [7]:
test_df

,User,Window_id,Text,Label,polarity,subjectivity,negativity,positivity,neutrality,compound,Vector
0,test_subject1445,0,"yeah he moved across the country, essentially...",1,0.631926,0.474791,0.110,0.170,0.720,0.99700,"[0.05043364316225052, 0.018346525728702545, 0...."
1,test_subject1445,1,"that is true, i never really thought about it...",1,0.648210,0.466057,0.117,0.171,0.712,0.99620,"[0.06412471830844879, 0.048445411026477814, 0...."
2,test_subject1445,2,i agree about the relationships evolving part...,1,0.642313,0.494077,0.141,0.159,0.700,0.96320,"[0.07285874336957932, 0.06897232681512833, 0.0..."
3,test_subject1445,3,good point. i just feel like i have screwed u...,1,0.635698,0.517881,0.137,0.196,0.667,0.99485,"[0.04998762533068657, 0.058237671852111816, -0..."
4,test_subject1445,4,"technically, not loving him. but i have been ...",1,0.624213,0.513609,0.132,0.198,0.670,0.99595,"[0.05251241847872734, 0.03347595036029816, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...
36749,subject9832,995,exactly scumbag christian girl.. i find myse...,0,0.625000,0.125000,0.144,0.000,0.856,0.18155,"[-0.04433127120137215, 0.10247306525707245, -0..."
36750,subject9832,996,scumbag christian girl.. i find myself doing ...,0,0.500000,0.000000,0.149,0.000,0.851,0.18155,"[-0.02481621876358986, 0.09474856406450272, 0...."
36751,subject9832,997,i find myself doing this everytime i walk thou...,0,0.500000,0.000000,0.000,0.000,1.000,0.50000,"[-0.008946556597948074, 0.07886570692062378, -..."
36752,subject9832,998,...snow days keep your children from school.....,0,0.500000,0.000000,0.000,0.000,1.000,0.50000,"[-0.010781817138195038, 0.144556924700737, -0...."


## Model part

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
#from transformers import AutoTokenizer

torch.manual_seed(seed)

#tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

class WritingWindowDataset(Dataset):
    def __init__(self, vectors, labels):

        self.labels = [label for label in labels]
        self.vectors = [vector for vector in vectors]
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_vectors(self, idx):
        # Fetch a batch of inputs
        return self.vectors[idx]

    def __getitem__(self, idx):

        batch_vectors = self.get_batch_vectors(idx)
        batch_y = self.get_batch_labels(idx)
        

        return batch_vectors, batch_y

In [9]:
#train_ds = WritingWindowDataset(train_sentence_embeddings, train_df['Label'])
#val_ds = WritingWindowDataset(val_sentence_embeddings, test_df['Label'])
full_ds = WritingWindowDataset(np.concatenate((train_sentence_embeddings, val_sentence_embeddings), axis=0), pd.concat([train_df['Label'], test_df['Label']]))
#torch.save(train_ds, "nn_train_dataset.pkl")
#torch.save(val_ds, "nn_val_dataset.pkl")



In [10]:
class LmNeuralNetwork(nn.Module):
    def __init__(self, trial):
        super(LmNeuralNetwork, self).__init__()
        #self.Lm = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
        #for param in self.Lm.parameters():
        #    param.requires_grad = False
        self.layers=[]
        n_layers = trial.suggest_int("n_layers", 1, 3)

        in_features = 768
        for i in range(n_layers):
            out_features = trial.suggest_int("n_units_l{}".format(i), 4, min(in_features,128))
            self.layers.append(nn.Linear(in_features, out_features))
            self.layers.append(nn.ReLU())
            p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5, step=0.05)
            self.layers.append(nn.Dropout(p))

            in_features = out_features
        self.layers.append(nn.Linear(in_features, 1))
        #self.classifier = nn.Sequential(*self.layers)
        self.cls_layers = torch.nn.ModuleList(self.layers)
    #def forward(self, input_id, mask):
    def forward(self, x):

        #_, output = self.Lm(input_ids= input_id, attention_mask=mask,return_dict=False)
        for layer in self.cls_layers:
            x = layer(x)
        #output=self.cls_layers(features)
        return torch.sigmoid(x)

#model = LmNeuralNetwork()

In [11]:
from sklearn.model_selection import StratifiedKFold
n_folds=5
skf = StratifiedKFold(n_splits=n_folds)
def objective(trial):
    
    user_label_df =train_df.drop_duplicates('User')
    users = user_label_df['User'].to_numpy()
    labels = user_label_df['Label'].to_numpy()

    #score = objective( trial, trainloader, valloader)
    params = {'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)}
    score=[]
    for fold, (user_train_ids, user_val_ids) in enumerate(skf.split(users,labels)):
        model= LmNeuralNetwork(trial)
        train_users = [users[f] for f in user_train_ids]
        val_users = [users[f] for f in user_val_ids]
        train_ids = train_df.index[train_df['User'].isin(train_users)].to_list()
        val_ids = train_df.index[train_df['User'].isin(val_users)].to_list()

        #train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        #test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
        train_data = torch.utils.data.Subset(full_ds, train_ids)
        val_data = torch.utils.data.Subset(full_ds, val_ids)

        trainloader = torch.utils.data.DataLoader(
                        train_data, 
                        batch_size=32, 
                        shuffle=True)
        valloader = torch.utils.data.DataLoader(
                        val_data,
                        batch_size=32,
                        shuffle=True)#, sampler=test_subsampler)
        print(f"Fold {fold}")
        res = train_eval(trial,model,params,10,trainloader,valloader)
        score.append(res)
    return np.mean(score)
       

In [12]:
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import f1_score
#from torchmetrics.functional import f1_score
BATCH_SIZE = 32

def train_eval(trial,model, params, epochs, train_dataloader, val_dataloader):

    #train, val = WritingWindowDataset(train_data), WritingWindowDataset(val_data)

    #train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    #val_dataloader = torch.utils.data.DataLoader(val_ds, batch_size=BATCH_SIZE)
   
    model = model.cuda()
    optimizer = AdamW(model.parameters(), lr=params['learning_rate'])#, lr= learning_rate)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.BCELoss()
    
            

    for epoch_num in range(epochs):
       
        total_loss_train = 0
        #train_preds = torch.tensor([], requires_grad=False).to(device)
        #train_targets = torch.tensor([], requires_grad=False).to(device)
        train_preds = np.array([])
        train_targets = np.array([])
        model.train()
        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            train_label = train_label.float()
            train_label = train_label.unsqueeze(1)
            features = train_input.to(device)
            

            output = model(features)
            batch_loss = criterion(output, train_label)
            total_loss_train += float(batch_loss.item())
            
            #train_preds = torch.cat((train_preds,torch.round(output).detach()),0)
            #train_targets=torch.cat((train_targets,train_label.detach()),0)
            train_preds = np.concatenate((train_preds,torch.round(output).detach().cpu().numpy().flatten()),axis=0)
            train_targets = np.concatenate((train_targets,train_label.detach().cpu().numpy().flatten()),axis=0)

            
            #acc = (train_y_pred_tag == train_label).sum().item()
            #total_acc_train += float(acc)
            #seen_train+=len(train_preds)
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        train_loss=total_loss_train/len(train_preds)
        train_f1 = f1_score(train_targets,train_preds)

        total_loss_val = 0
        val_preds = np.array([])
        val_targets = np.array([])
        model.eval()
        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                val_label = val_label.float()
                val_label = val_label.unsqueeze(1)

                #mask = val_input['attention_mask'].to(device)
                #input_id = val_input['input_ids'].squeeze(1).to(device)
                features = val_input.to(device)

                output = model(features)

                batch_loss = criterion(output, val_label)
                total_loss_val += float(batch_loss.item())
                val_preds = np.concatenate((val_preds,torch.round(output).detach().cpu().numpy().flatten()),axis=0)
                val_targets = np.concatenate((val_targets,val_label.detach().cpu().numpy().flatten()),axis=0)

                
        val_f1 = f1_score(val_targets,val_preds)
        val_loss=total_loss_val/len(val_preds)
        
        if epoch_num in [0,4,9]:
            print(
                    f'Epochs: {epoch_num + 1 } | Train Loss: {train_loss: .4f} \
                    | Train F1: {train_f1: .4f} \
                    | Val Loss: {val_loss: .4f} \
                    | Val F1: {val_f1: .4f}')

        #trial.report(val_f1, epoch_num)

        #if trial.should_prune():
        #    raise optuna.exceptions.TrialPruned()

    return val_f1


In [13]:
import optuna
import joblib
study = optuna.create_study(study_name=f"LM_NN_Tuning_{ROLLING_WINDOW_SIZE}_{LM_MODEL}",direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.NopPruner())
#study = joblib.load(f"hyperparameterstudy{ROLLING_WINDOW_SIZE}.pkl")
for i in range(20):
    print(f"trial {i}")
    study.optimize(objective, n_trials=1, timeout=(60*60*10), gc_after_trial=True)
    joblib.dump(study, f"hyperparameterstudy_{ROLLING_WINDOW_SIZE}_{LM_MODEL}.pkl")


[I 2022-07-15 07:25:37,358] A new study created in memory with name: LM_NN_Tuning_10_all-mpnet-base-v2


trial 0
Fold 0


100%|██████████| 4429/4429 [00:44<00:00, 99.95it/s] 


Epochs: 1 | Train Loss:  0.0160                     | Train F1:  0.6773                     | Val Loss:  0.0153                     | Val F1:  0.7051


100%|██████████| 4429/4429 [00:12<00:00, 350.45it/s]


...

Epochs: 1 | Train Loss:  0.0206                     | Train F1:  0.6018                     | Val Loss:  0.0206                     | Val F1:  0.5071


100%|██████████| 4291/4291 [00:12<00:00, 333.25it/s]


Epochs: 5 | Train Loss:  0.0141                     | Train F1:  0.7540                     | Val Loss:  0.0208                     | Val F1:  0.6233


100%|██████████| 4291/4291 [00:12<00:00, 334.77it/s]
[I 2022-07-15 12:07:42,885] Trial 19 finished with value: 0.6928846911192406 and parameters: {'learning_rate': 2.5781611868847455e-05, 'n_layers': 2, 'n_units_l0': 20, 'dropout_l0': 0.45, 'n_units_l1': 10, 'dropout_l1': 0.5}. Best is trial 2 with value: 0.6940657012575293.


Epochs: 10 | Train Loss:  0.0129                     | Train F1:  0.7747                     | Val Loss:  0.0215                     | Val F1:  0.6291


In [14]:
#study = joblib.load(f"NN_hyperparameterstudy10.pkl")

#study.optimize(objective, n_trials=5, timeout=(60*60*2), gc_after_trial=True)


In [15]:
joblib.dump(study, f"NN_hyperparameterstudy{ROLLING_WINDOW_SIZE}.pkl")


['NN_hyperparameterstudy10.pkl']

In [16]:
#load
import joblib
study = joblib.load(f"NN_hyperparameterstudy10.pkl")
print("Best trial until now:")
print(" Value: ", study.best_trial.value)
print(" Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

Best trial until now:
 Value:  0.6940657012575293
 Params: 
    learning_rate: 4.982872361890929e-05
    n_layers: 3
    n_units_l0: 6
    dropout_l0: 0.5
    n_units_l1: 6
    dropout_l1: 0.30000000000000004
    n_units_l2: 5
    dropout_l2: 0.5


In [17]:
import optuna
fig = optuna.visualization.plot_slice(study)
fig.show()

In [18]:
#study.optimize(objective, n_trials=2, gc_after_trial=True)
#joblib.dump(study, f"hyperparameterstudy{ROLLING_WINDOW_SIZE}.pkl")
